In [35]:
import pandas as pd
import glob
import pandas_ta as ta

headers = ['Date','Time','Open','High','Low','Close','Volume']
path = "C:\\Users\\desma\\Downloads\\jupyter work\\DATA\\"
#set up so i could run multiple datasets if wanted
filenames = glob.glob(path + "QQQ15m.txt")
for file in filenames:
    print("\nReading file = ", file)
    df = pd.read_csv(file, header = None)
    df.columns=headers
#df['Datetime'] = df['Date'] + ' ' + df['Time']
df.index = pd.to_datetime(df['Date'] + ' ' + df['Time'])
df['VWAP'] = ta.vwap(df.High, df.Low, df.Close, df.Volume)
df.reset_index(inplace=True)
dfLen = len(df)


Reading file =  C:\Users\desma\Downloads\jupyter work\DATA\QQQ15m.txt


In [36]:
engulfSignal = [0]*dfLen
signal = [0]*dfLen


#Engulfing
for row in range(1, dfLen): 
    diffMin = abs(df.Open[row-1]-df.Close[row-1]) * 1.25
    bodyDiff = abs(df.Open[row]-df.Close[row]) 
    #BEARISH reversal: signal = 2
    if(bodyDiff>diffMin and df.Open[row-1]<df.Close[row-1] and df.Open[row]>df.Close[row] 
       and df.Open[row]>df.Close[row-1] and df.Close[row]<df.Open[row-1]):\
        engulfSignal[row] = 2 
            
    #BULLISH reversal: signal = 1
    elif(bodyDiff>diffMin and df.Open[row-1]>df.Close[row-1] and df.Open[row]<df.Close[row] 
       and df.Open[row]<df.Close[row-1] and df.Close[row]>df.Open[row-1]):
        engulfSignal[row] = 1 
df['Engulfing'] = engulfSignal


#VWAP
for row in range(1, dfLen):
    close = df.Close[row]
    VWAP = df.VWAP[row]
    engSig = df.Engulfing[row]
    #long position (above VWAP and bullish engulfing) --> signal == 1
    if close>VWAP and engSig==1:
        signal[row] = 1
        
    #short position (below VWAP and bearish engulfing) --> signal == 2
    elif close<VWAP and engSig==2:
        signal[row] = 2
        
df['Signal'] = signal
df['Signal'].value_counts()

0    22559
1      574
2      541
Name: Signal, dtype: int64

In [37]:
def SIGNAL():
    return df.Signal

In [38]:
from backtesting import Strategy, Backtest
import numpy as np

class myStrat(Strategy):
    def init(self):
        super().init()
        self.signal1=self.I(SIGNAL)     
        
    def next(self):
        super().next()
        trailSL = self.data.Close[-1]*.005
        for trade in self.trades:
            if trade.is_long:
                trade.sl = max(trade.sl or -np.inf, self.data.Close[-1] - trailSL)
            else:
                trade.sl = min(trade.sl or np.inf, self.data.Close[-1] + trailSL)
        
        if self.signal1==1 and len(self.trades)==0: #Long Signal
            self.buy()
            
        if self.signal1==2 and len(self.trades)==0: #Short Signal
            self.sell()
            
bt = Backtest(df, myStrat, cash=100_000, margin = 1/1, commission=.000)
stat = bt.run()
stat
#bt.plot(show_legend=False, superimpose = True)

C:\Users\desma\AppData\Local\Temp/ipykernel_4932/1308077041.py:24: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df, myStrat, cash=100_000, margin = 1/1, commission=.000)


Start                                     0.0
End                                   23673.0
Duration                              23673.0
Exposure Time [%]                    52.51753
Equity Final [$]                   80601.4891
Equity Peak [$]                   104051.2379
Return [%]                         -19.398511
Buy & Hold Return [%]               -9.083096
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -26.109165
Avg. Drawdown [%]                   -1.508651
Max. Drawdown Duration                21475.0
Avg. Drawdown Duration                  944.6
# Trades                                599.0
Win Rate [%]                         36.72788
Best Trade [%]                       3.642165
Worst Trade [%]                     -1.818541
Avg. Trade [%]                    